In [1]:
import pytesseract
import numpy as np
import cv2
import matplotlib.pyplot as plt

gojiseo={}

def _gojiseo():
### 날짜 정보
    pytesseract.pytesseract.tesseract_cmd = r'/opt/homebrew/bin/tesseract'

    config = ('-l kor+eng --oem 3 --psm 4') 
    
    def gray_scale(image):
    # img_gray=cv2.cvtColor(img, cv2.COLOR_BAYER_BG2GRAY) // 왜 오류나는지 모르겠음
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    def deskew(image): # 숫자 이미지를 읽어 기울어진 이미지 보정
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        gray = cv2.bitwise_not(gray)
        thresh = cv2.threshold(gray, 0, 255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        coords = np.column_stack(np.where(thresh > 0))
        angle = cv2.minAreaRect(coords)[-1]
        if angle < -45:
            angle = -(90 + angle)
        else:
            angle = -angle
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        rotated = cv2.warpAffine(image, M, (w, h),
        flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)    
        return rotated

    deskew = deskew(img)
    gray = gray_scale(deskew)
        
    x_indivi=770; y_indivi=20; w_indivi=450; h_indivi=340    # roi 좌표(개별관리비)
    x_date=480; y_date=450; w_date=220; h_date=270     # roi 좌표(날짜 정보)
    x_price=480; y_price=670; w_price=230; h_price=270     # roi 좌표(이번 달 총금액)


    roi1 = gray[x_indivi:x_indivi+h_indivi , y_indivi:y_indivi+w_indivi]  # roi 지정
    roi2 = gray[x_date:x_date+h_date , y_date:y_date+w_date]  # roi 지정
    roi3 = gray[x_price:x_price+h_price, y_price:y_price+w_price]  # roi 지정

    cv2.rectangle(roi1, (0,0), (w_indivi-1, h_indivi-1), (0,0,255) ,2) # roi 전체에 사각형 그리기
    cv2.rectangle(roi2, (0,0), (w_date-1, h_date-1), (0,0,255) ,2) # roi 전체에 사각형 그리기
    cv2.rectangle(roi3, (0,0), (w_price-1, h_price-1), (0,0,255) ,2) # roi 전체에 사각형 그리기

    text_indivi=pytesseract.image_to_string(roi1,config=config)
    text_date=pytesseract.image_to_string(roi2,config=config)
    text_price=pytesseract.image_to_string(roi3,config=config)


##################################

    start_date=text_date.find('월분')+5
    date_l=text_date[start_date:start_date+7].strip().strip('、').strip(',').strip('.').split("-")
    global date_this
    date_this=date_l[0]+date_l[1]
### 전기
    start_use_e = text_indivi.find('전기')+2
    elec_use = text_indivi[start_use_e:start_use_e+4].strip()
    start_price_e = text_indivi.find('kw')+2
    elec_price = text_indivi[start_price_e:start_price_e+25].strip()
    elec_price = elec_price.replace(",",'').strip()

### 수도
    start_use_w = text_indivi.find('수도')+2
    water_use = text_indivi[start_use_w:start_use_w+3].strip()
    start_price_w = text_indivi.find('수도')+10
    water_price = text_indivi[start_price_w:start_price_w+12].strip()
    water_price = water_price.replace(",",'').strip()

### 총금액 --> !!위치로 따온거라 수정해야 할수도 있음!!
    splits=text_price.splitlines()
    price_this=splits[2]
    total_price = price_this.replace(",",'').strip()

### 공용관리비 정보
    start_use_public = text_indivi.find('공용소계')+15
    public_price = text_indivi[start_use_public:start_use_public+15].strip()
    public_price = public_price.replace(",",'').strip()

# print("날짜 정보(yyyymm): ", date_this,'\n')
# print("전기 사용량:", elec_use,"kw")
# print("전기 요금:", elec_price,'원\n')
# print("수도 사용량:", water_use,"m3")
# print("수도 요금:", water_price,'원\n')
# print("공용 관리비: ", public_price,'원\n')
# print("총 요금:", total_price,"원")

    gojiseo=[date_this, water_price, water_use, elec_price, elec_use, public_price, total_price]
    
    return gojiseo


In [2]:
### 다음 달 금액 예측하기 (테스트용 데이터)

from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from datetime import datetime
from dateutil.relativedelta import *

test_elect_price=[35000,38010,28150,28800,27160,39330,37470,51389,42140,34740,36410,33390,40610] # 41280
test_water_price=[24990,28430,29780,24390,25740,25740,29780,28430,28430,32470,27090,28430,23050] # 2574
test_total_price=[173560,185120,171060,165720,165560,175850,157110,167680,159940,161840,153840,157420,161990] # 165170

def rbf_predict(total_price):
    months=[]
    j=1
    for j in range(len(total_price)):
        months.append([int(j+1)])
        j=j+1

    x_train_t, x_test_t, y_train_t ,y_test_t = train_test_split(months,total_price,test_size=0.2, random_state=777)

    rbf_svr_t1 = SVR(kernel='rbf', C=20000, gamma=0.3)
    rbf_svr_t1.fit(x_train_t, y_train_t)


    nextMonth = [[len(months)+1]]
    
    n_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-1)).strftime("%Y%m")
    b_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-2)).strftime("%Y%m")
    bb_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-13)).strftime("%Y%m")
    
    total_predict = rbf_svr_t1.predict(nextMonth)
    total_predict = int(total_predict[0])
    
    rising_rate=(rbf_svr_t1.predict(nextMonth)/test_total_price[-11])*100
    comparison=round(rising_rate[0],2)
    return total_predict, comparison

user_data_predict=rbf_predict(test_total_price)
print(user_data_predict)


(166957, 97.6)


In [ ]:
import pymysql
import logging
import warnings
warnings.filterwarnings("ignore")

work_db = pymysql.connect(
    user='root', 
    passwd='admin', 
    host='168.131.198.17', 
    db='eggo_db', 
#     charset='utf8'
)

work_db

In [13]:
def sendInfo():
    cursor = work_db.cursor(pymysql.cursors.DictCursor)
    sql = 'INSERT INTO gojiseo(date,waterFee,waterUse,electFee,electUse,publicFee,totalFee) values("202103","29780","26","28150","258","107020","171060")'
    cursor.execute( sql )
    work_db.commit()
sendInfo()

["202104","24390","22","28800","262","106420","165720"]
["202105","25740","23","27160","252","106550","165560"]
["202106","25740","23","39330","326","104650","175850"]
["202107","29780","26","37470","340","83770","157110"]
["202108","28430","25","51380","447","81760","167680"]
["202109","28430","25","42140","366","83240","159940"]
["202110","32470","28","34740","292","88520","161840"]
["202111","27090","24","36410","302","84190","153840"]
["202112","28430","25","33390","284","89490","157420"]
["202201","23050","21","40610","327","91910","161990"]

OperationalError: (2006, "MySQL server has gone away (BrokenPipeError(32, 'Broken pipe'))")

In [5]:
def readInfo():
    cursor = work_db.cursor(pymysql.cursors.DictCursor)
    sql = 'SELECT * FROM gojiseo'
    cursor.execute( sql )
#     work_db.commit()
    c= cursor.fetchall()
    for x in c:
        print(x)
readInfo()

In [6]:
# readImage() 서버로 이미지 받아오기
from datetime import datetime
from dateutil.relativedelta import *

# def sendInfo():
#     cursor = work_db.cursor()
#     sql = 'INSERT INTO gojiseo(date, electUse, waterUse, electFee, waterFee, publicFee, totalFee) values(date_this, elec_use, water_use, elec_price, water_price, public_price, total_price)'
#     cursor.execute( sql )
#     work_db.commit()
    
# def pieInfo():
#     cursor = work_db.cursor()
#     sql = 'INSERT INTO pieData(electFee, waterFee, etcIndividualFee, publicFee, totalFee) values(elec_price, wat_price, etc_indivi_price, public_price, total_price)'
#     cursor.execute( sql )
#     work_db.commit()
###############

# b_month: 전전달 / bb_month: 전전달까지 12개 데이터 / n_month: 이전달 나올 고지서 예측

    
# user_data_predict=rbf_predict(test_total_price)
    
# def sendRealtimePredict():
#     cursor = data_db.cursor()
#     sql = 'INSERT INTO realTimePredict() values()'
#     cursor.execute( sql )
#     work_db.commit()  
    
    
    
# readDanjiImform() 지역코드, 아파트이름 받아오기
# api는 시간 많이 걸리니까 메시지 출력하기 ('정보를 가져오고 있습니다.')

# def sendDanjiComparison():
#     cursor = data_db.cursor()
#     sql = 'INSERT INTO danjiUserPredict(pk, date, danjiPrice, userPredict) values(4, danji_predict(sidoCode, aptName)[0],danji_predict(sidoCode, aptName)[1],danji_predict(sidoCode, aptName)[2])'

In [7]:
from flask import Flask
import pandas as pd 
import pymysql
import PIL
from PIL import Image
import io


In [8]:
filename = 'data_nxny.xlsx'


In [9]:
app = Flask(__name__)
global img

In [10]:
@app.route('/')
def hello_world():
    print('d')
    return 'Hello'

@app.route('/getPhoto')
def getPhoto():
    print("==GET Photo==")
    try : 

        work_db = pymysql.connect(user='root',password='admin',host = '168.131.198.17',db='eggo_db',charset='utf8')
        cursor = work_db.cursor()
        sql = 'SELECT * FROM photo limit 1'
        cursor.execute(sql)
        photo_data = cursor.fetchall()
        data_io = io.BytesIO(photo_data[0][0])
        print(data_io)
        global img 
        img = Image.open(data_io)   
        img=np.array(img)

        _gojiseo()
        date,wf,wu,ef,eu,pf,tf=_gojiseo()[0],_gojiseo()[1],_gojiseo()[2],_gojiseo()[3],_gojiseo()[4],_gojiseo()[5],_gojiseo()[6]
        print(date,wf,wu,ef,eu,pf,tf)
        sql = 'INSERT INTO gojiseo(date,waterFee,waterUse,electFee,electUse,publicFee,totalFee) VALUES(date,wf,wu,ef,eu,pf,tf)'
        cursor.execute(sql)
        work_db.commit()  
        work_db.close()
        return "success"
    
    except Exception as e:
        print(e)
        return "fail"

    
n_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-1)).strftime("%Y%m")
b_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-2)).strftime("%Y%m")
bb_month = (datetime(datetime.today().year,datetime.today().month,1)+relativedelta(months=-13)).strftime("%Y%m")

n_month=str(n_month)
b_month=str(b_month)
bb_month=str(bb_month)

@app.route('/getUserdata')
def readUserData(): #사용자 데이터 이용한 다음달 납부금액 예측
    print("==GET UserData==")
    try:
        work_db = pymysql.connect(user='root',password='admin',host = '168.131.198.17',db='eggo_db',charset='utf8')
        cursor = work_db.cursor()
        sql = "SELECT totalFee FROM gojiseo WHERE date >= '%s' AND date <= '%s'" %(b_month,bb_month)
        cursor.execute( sql )
        work_db.commit()
        total=cursor.fetchall()
        print(total)
         
        user_data_predict=rbf_predict(total)
        total_predict=str(user_data_predict[0])
        rising_rate=str(user_data_predict[1])
        sql="INSERT INTO userPredict(date,totalFee,rateOfChange) VALUES(n_month,total_predict,rising_rate)"
        curosr.execute(sql)
        work_db.commit()
        work_db.close()
        return "success"
    except Exception as e:
        print(e)
        return "fail"
   
    
@app.route('/getAddress')
def getAddress():
    print("==GET Address==")
    try:
        work_db = pymysql.connect("db")
        cursor = work_db.cursor()
        
        sql = 'SELECT * FROM address limit 1'
        cursor.execute(sql)
        address = cursor.fetchall()
        
        region1 = address[0][0]
        region2 = address[0][1]
        region3 = address[0][2]
        
        df = pd.read_excel('data_nxny.xlsx', engine='openpyxl')
        region_data = df[(df['1단계']==region1) & (df['2단계']==region2) & (df['3단계']==region3)]
        nx = (region_data['격자 X'].values)[0]
        ny = (region_data['격자 Y'].values)[0]
        
        sql = '''INSERT INTO nxny VALUES(%s,%s)''' %(nx,ny)
        cursor.execute(sql, (nx,ny))
        
        work_db.commit()
        work_db.close()
        
        return "success"
    except Exception as e:
        print(e)
        return "fail"      
    
@app.route('/getRealTimeData')
def getRealTimeData():
    print("==GET RealTimeData==")
    try:
        work_db = pymysql.connect("db")
        cursor = work_db.cursor()
        
        sql = 'SELECT * FROM weatherData limit 1'
        cursor.execute(sql)
        weather = cursor.fetchall()
        print(weather[0])
        
        ##실시간 요금 딥러닝한거 추가##
        sql = '''INSERT INTO realTimePredict VALUES(5000)'''
        cursor.execute(sql)
        work_db.commit()
        work_db.close()
        return "success"
    except Exception as e:
        print(e)
        return "fail"

        
@app.route('/getTest')
def getTest():
    return "success"

In [ ]:
# 모든 host에서 접근 가능
if __name__ == '__main__':
    app.run(host='0.0.0.0',port='5001') #host='0.0.0.0'

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on all addresses.
 * Running on http://168.131.213.46:5001/ (Press CTRL+C to quit)


==GET Photo==


168.131.198.17 - - [23/May/2022 16:57:00] "GET /getPhoto HTTP/1.1" 200 -


202202 25740 23 41280 331 91590 165170
(1054, "Unknown column 'wf' in 'field list'")
